In [42]:
# So we can load files from other sub-directories, e.g. datasets.
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
from torch import nn
import utils.utils as utils
import json
import numpy as np
from scipy.special import softmax
from sklearn.linear_model import LogisticRegression, SGDClassifier

import calibration as cal


In [8]:
print(f'cuda is available: {torch.cuda.is_available()}')
torch.cuda.empty_cache()

cuda is available: True


In [9]:
USE_CUDA = True
cuda = torch.device('cuda') 
batch_size, num_workers = 32, 16

In [4]:
def load_model(config_path, checkpoint_path):
    with open(config_path) as f:
        config = json.load(f)
    net = utils.initialize(config['model'])
    if USE_CUDA:
        net = net.cuda()
    net.new_last_layer(config['num_classes'])
    utils.load_ckp(checkpoint_path, net)
    return net

# Load datasets.
def load_test_dataset(config, idx, split_arg_name='split', split='val'):
    test_config = config['test_datasets'][idx]
    test_config['args'][split_arg_name] = split
    print(test_config['name'], test_config)
    if 'transforms' not in test_config:
        test_config['transforms'] = config['default_test_transforms']
    test_data = utils.init_dataset(test_config)
    test_loader = torch.utils.data.DataLoader(
        test_data, batch_size=batch_size,
        shuffle=False, num_workers=num_workers)
    return test_data, test_loader

def get_features_labels(net, loader):
    net.cuda()
    net.eval()
    feature_model = nn.Sequential(*list(net._model.children())[:-1])
    features_list, labels_list = [], []
    with torch.no_grad():
        for data in loader:
            images, labels = data
            images, labels = images.cuda(), labels.cuda()
            features = feature_model(images)
            features_list.append(features.detach().cpu().numpy())
            labels_list.append(labels.detach().cpu().numpy())
    features = np.squeeze(np.concatenate(features_list))
    labels = np.concatenate(labels_list)
    return features, labels

def get_acc(preds, labels):
    return np.mean(preds == labels)

def make_none_list(rs, cs):
    return [[None] * cs for _ in range(rs)]

In [5]:
# Given a network how to get representations
def get_model_representations(
    config_paths, checkpoint_paths, model_names, loader_names, loader_indices, split_arg_names, split_names):
    M, L = len(model_names), len(loader_names)
    models = []
    for m in range(M):
        models.append(load_model(config_paths[m], checkpoint_paths[m]))

    with open(config_paths[0]) as f:
        config = json.load(f)
    loaders = []
    for l in range(L):
        _, loader = load_test_dataset(config, loader_indices[l], split_arg_names[l], split_names[l])
        loaders.append(loader)
    features, labels = make_none_list(M, L), make_none_list(M, L)
    for m in range(M):
        for l in range(L):
            features[m][l], labels[m][l] = get_features_labels(models[m], loaders[l])    
    return features, labels
        
def get_test_accs(features, labels, C=1.0):
    M, L = len(features), len(features[0])
    assert(L == 2)  # Only support train on first, validate on second for now
    clfs = []
    preds = []
    accs = []
    for m in range(M):
        cur_clf = LogisticRegression(random_state=0, C=C).fit(features[m][0], labels[m][0])
        cur_preds = cur_clf.predict(features[m][1])
        cur_acc = get_acc(cur_preds, labels[m][1])
        print(cur_acc)
        clfs.append(cur_clf)
        preds.append(cur_preds)
        accs.append(cur_acc)
    return clfs, preds, accs


In [10]:
# Living17 MOCO

config_paths = [
    '../logs/breeds_moco_ft_0.003/config.json',
]

checkpoint_paths = [
    '../logs/breeds_moco_ft_0.003/checkpoints/ckp_0',
]

model_names = ['original_model']
loader_names = ['l17_trg_train', 'l17_trg_val', 'l17_src_train', 'l17_src_val']
split_names = ['train', 'val', 'train', 'val']
split_arg_names = ['split', 'split', 'split', 'split']
loader_indices = [1, 1, 0, 0]  # Relative to the config file in the first config path in config_paths.

features, labels = get_model_representations(
    config_paths, checkpoint_paths, model_names, loader_names, loader_indices,
    split_arg_names, split_names)

target_val_living {'name': 'target_val_living', 'max_test_examples': 1000, 'classname': 'datasets.breeds.Breeds', 'args': {'source': False, 'split': 'train', 'breeds_name': 'living17', 'root': '/u/scr/nlp/imagenet/'}}
target_val_living {'name': 'target_val_living', 'max_test_examples': 1000, 'classname': 'datasets.breeds.Breeds', 'args': {'source': False, 'split': 'val', 'breeds_name': 'living17', 'root': '/u/scr/nlp/imagenet/'}, 'transforms': [{'classname': 'torchvision.transforms.Resize', 'args': {'size': [224, 224]}}, {'classname': 'torchvision.transforms.ToTensor'}, {'classname': 'torchvision.transforms.Normalize', 'args': {'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225]}}]}
source_val_living {'name': 'source_val_living', 'max_test_examples': 1000, 'classname': 'datasets.breeds.Breeds', 'args': {'source': True, 'split': 'train', 'breeds_name': 'living17', 'root': '/u/scr/nlp/imagenet/'}}
source_val_living {'name': 'source_val_living', 'max_test_examples': 1000, 'classna

In [30]:
# Entity 30 MOCO

config_paths = [
    '../logs/entity30_moco_ft_0.003/config.json',
]

checkpoint_paths = [
    '../logs/entity30_moco_ft_0.003/checkpoints/ckp_0',
]

model_names = ['original_model']
loader_names = ['e30_trg_train', 'e30_trg_val', 'e30_src_train', 'e30_src_val']
split_names = ['train', 'val', 'train', 'val']
split_arg_names = ['split', 'split', 'split', 'split']
loader_indices = [1, 1, 0, 0]  # Relative to the config file in the first config path in config_paths.

e30_features, e30_labels = get_model_representations(
    config_paths, checkpoint_paths, model_names, loader_names, loader_indices,
    split_arg_names, split_names)

target_val_entity {'name': 'target_val_entity', 'max_test_examples': 1000, 'classname': 'datasets.breeds.Breeds', 'args': {'source': False, 'split': 'train', 'breeds_name': 'entity30', 'root': '/u/scr/nlp/imagenet/'}}
target_val_entity {'name': 'target_val_entity', 'max_test_examples': 1000, 'classname': 'datasets.breeds.Breeds', 'args': {'source': False, 'split': 'val', 'breeds_name': 'entity30', 'root': '/u/scr/nlp/imagenet/'}, 'transforms': [{'classname': 'torchvision.transforms.Resize', 'args': {'size': [224, 224]}}, {'classname': 'torchvision.transforms.ToTensor'}, {'classname': 'torchvision.transforms.Normalize', 'args': {'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225]}}]}
source_val_entity {'name': 'source_val_entity', 'max_test_examples': 1000, 'classname': 'datasets.breeds.Breeds', 'args': {'source': True, 'split': 'train', 'breeds_name': 'entity30', 'root': '/u/scr/nlp/imagenet/'}}
source_val_entity {'name': 'source_val_entity', 'max_test_examples': 1000, 'classna

In [ ]:
# # On CIFAR, Im-n-C10

# config_paths = [
#     '../logs/imnet_cifar_all_ft_0d01/config.json',
# ]

# checkpoint_paths = [
#     '../logs/imnet_cifar_all_ft_0d01/checkpoints/ckp_0',
# ]

# model_names = ['original_model']
# loader_names = ['e30_trg_train', 'e30_trg_val', 'e30_src_train', 'e30_src_val']
# split_names = ['train', 'val', 'train', 'val']
# split_arg_names = ['split', 'split', 'split', 'split']
# loader_indices = [4, 4, 0, 0]  # Relative to the config file in the first config path in config_paths.

# e30_features, e30_labels = get_model_representations(
#     config_paths, checkpoint_paths, model_names, loader_names, loader_indices,
#     split_arg_names, split_names)

In [20]:
np.std(features[0][2])

0.021199878

In [53]:
# Train logistic regression
def test_log_reg(features, labels, source_idx, test_indices, C=1.0, sgd=False):
    M, L = len(features), len(features[0])
    m = 0
    if not sgd:
        cur_clf = LogisticRegression(random_state=0, C=C).fit(features[m][source_idx], labels[m][source_idx])
    else:
        cur_clf = SGDClassifier(max_iter=10, loss='log', random_state=0, alpha=0.0, eta0=0.01, learning_rate='constant').fit(features[m][source_idx], labels[m][source_idx])
    for l in test_indices:
        cur_preds = cur_clf.predict(features[m][l])
        cur_acc = get_acc(cur_preds, labels[m][l])
        print(cur_acc)
    return cur_clf


In [54]:
# trg_clf = test_log_reg(features, labels, source_idx=0, test_indices=[1, 3], C=1.0)
src_clf = test_log_reg(features, labels, source_idx=2, test_indices=[1, 3], C=10.0, sgd=True)

0.7894117647058824
0.9317647058823529


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


In [48]:
e30_trg_clf = test_log_reg(e30_features, e30_labels, source_idx=0, test_indices=[1, 3], C=1.0, sgd=True)
e30_src_clf = test_log_reg(e30_features, e30_labels, source_idx=2, test_indices=[1, 3], C=10.0, sgd=True)

KeyboardInterrupt: 

In [35]:
# Given two feature sets, make H-delta-H divergence dataset?
def make_hdeltah_labels(xs1, xs2):
    n1, n2 = len(xs1), len(xs2)
    ys = [0] * n1
    ys.extend([1] * n2)
    return np.concatenate([xs1, xs2]), np.array(ys)

def test_hdeltah(features, C=1.0):
    l17_hdeltah_xs_train, l17_hdeltah_ys_train = make_hdeltah_labels(features[0][2], features[0][0])
    l17_hdeltah_xs_val, l17_hdeltah_ys_val = make_hdeltah_labels(features[0][3], features[0][1])
    cur_clf = LogisticRegression(random_state=0, C=C).fit(l17_hdeltah_xs_train, l17_hdeltah_ys_train)
    cur_preds = cur_clf.predict(l17_hdeltah_xs_val)
    cur_acc = get_acc(cur_preds, l17_hdeltah_ys_val)
    return cur_clf, cur_acc
    

In [36]:
cur_clf, cur_acc = test_hdeltah(features, C=10.0)
print(cur_acc)

/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


NameError: name 'cur_avv' is not defined

In [38]:
cur_clf, cur_acc = test_hdeltah(e30_features, C=10.0)
print(cur_acc)

/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.804


In [39]:
import pickle
pickle.dump((features, labels), open("../logs/l17_moco_features_labels", "wb"))
pickle.dump((e30_features, e30_labels), open("../logs/e30_moco_features_labels", "wb"))

In [ ]:
features, labels = pickle.load(open("../logs/l17_moco_features_labels", "rb"))
e30_features, e30_labels = pickle.load(open("../logs/e30_moco_features_labels", "rb"))